# Requerimiento de recursos

## Librerias necesarias

In [127]:
import pandas as pd
import os
import pickle
import xlsxwriter
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import re
from typing import Dict, List, Any
import itertools
from firebase_admin import credentials, firestore
import excel2img
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import os
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from google.oauth2 import service_account
from google.cloud import firestore
import json
from datetime import date, timedelta
import calendar
import locale
import math
import random
from typing import List, Dict, Optional
from jinja2 import Template, Environment
from xhtml2pdf import pisa
import locale, calendar

In [128]:
locale.setlocale(locale.LC_TIME, 'es_PE.UTF-8')  

'es_PE.UTF-8'

## Datos del proyecto en análisis

In [129]:
proyecto="sibayo"
mes=4
anio=2025

In [130]:
mes_recursos_solicitados=mes+1

## Datos de Firestore

In [131]:
## Datos de # 1. Busca el .env en el directorio actual o en cualquiera de los padres
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError("No se encontró ningún archivo .env en este directorio ni en sus padres.")
load_dotenv(dotenv_path)

# 2. Define el root del proyecto como la carpeta que contiene el .env
project_root = Path(dotenv_path).parent

# 3. Obtén la ruta relativa de las credenciales desde la variable de entorno
rel_cred_path = os.getenv("FIRESTORE_CREDENTIALS")
if not rel_cred_path:
    raise RuntimeError("No existe la variable FIRESTORE_CREDENTIALS en el .env")

# 4. Construye la ruta absoluta al JSON
cred_path = Path(rel_cred_path)
if not cred_path.is_absolute():
    cred_path = (project_root / cred_path).resolve()

if not cred_path.exists():
    raise FileNotFoundError(f"No existe el archivo de credenciales en: {cred_path}")

# 5. Carga las credenciales y crea el cliente de Firestore
credentials = service_account.Credentials.from_service_account_file(str(cred_path))
client = firestore.Client(credentials=credentials, project=credentials.project_id)

# 6. Prueba que funcione
print("Colecciones disponibles:", [c.id for c in client.collections()])

Colecciones disponibles: ['rutinarios']


In [132]:
db=firestore.Client(credentials=credentials, project=credentials.project_id)

In [133]:
# 4. Define la ruta a tu documento anidado
colec_raiz = "rutinarios"
doc_proyecto = proyecto     # puede ser tu variable proyecto
colec_valoriz = "presupuestos"
id_valoriz = "desembolsos"  

doc_ref = (
    db
    .collection(colec_raiz)
    .document(doc_proyecto)
    .collection(colec_valoriz)
    .document(id_valoriz)
)

data_mantenimiento_snapshot=doc_ref.get()

if not data_mantenimiento_snapshot.exists:
    print(f"El documento {doc_ref.path} no existe.")

data_mantenimiento= data_mantenimiento_snapshot.to_dict()
print(data_mantenimiento)

{'cronograma_desembolsos': {'10': {'mantenimiento_sin_igv': 3535.78, 'gastos_operativos': 431.19, 'mantenimiento_con_igv': 4311.93, 'mantenimiento_con_go': 4743.12, 'igv': 776.15}, '9': {'mantenimiento_sin_igv': 3421.72, 'gastos_operativos': 417.28, 'mantenimiento_con_igv': 4172.83, 'mantenimiento_con_go': 4590.11, 'igv': 751.11}, '6': {'mantenimiento_sin_igv': 40013.59, 'gastos_operativos': 4879.71, 'mantenimiento_con_igv': 48797.06, 'mantenimiento_con_go': 53676.77, 'igv': 8783.47}, '5': {'mantenimiento_sin_igv': 40127.65, 'gastos_operativos': 4893.62, 'mantenimiento_con_igv': 48936.16, 'mantenimiento_con_go': 53829.78, 'igv': 8808.51}, '7': {'mantenimiento_sin_igv': 38385.18, 'gastos_operativos': 4681.12, 'mantenimiento_con_igv': 46811.2, 'mantenimiento_con_go': 51492.32, 'igv': 8426.02}, '12': {'mantenimiento_sin_igv': 1140.58, 'gastos_operativos': 139.09, 'mantenimiento_con_igv': 1390.95, 'mantenimiento_con_go': 1530.05, 'igv': 250.37}, '4': {'mantenimiento_sin_igv': 1824.92, 'gas

In [134]:
cronograma_desembolsos = data_mantenimiento.get("cronograma_desembolsos", {})
if not cronograma_desembolsos:
    print("No se encontraron datos de cronograma de desembolsos.")
print("Cronograma de desembolsos:", cronograma_desembolsos)

Cronograma de desembolsos: {'10': {'mantenimiento_sin_igv': 3535.78, 'gastos_operativos': 431.19, 'mantenimiento_con_igv': 4311.93, 'mantenimiento_con_go': 4743.12, 'igv': 776.15}, '9': {'mantenimiento_sin_igv': 3421.72, 'gastos_operativos': 417.28, 'mantenimiento_con_igv': 4172.83, 'mantenimiento_con_go': 4590.11, 'igv': 751.11}, '6': {'mantenimiento_sin_igv': 40013.59, 'gastos_operativos': 4879.71, 'mantenimiento_con_igv': 48797.06, 'mantenimiento_con_go': 53676.77, 'igv': 8783.47}, '5': {'mantenimiento_sin_igv': 40127.65, 'gastos_operativos': 4893.62, 'mantenimiento_con_igv': 48936.16, 'mantenimiento_con_go': 53829.78, 'igv': 8808.51}, '7': {'mantenimiento_sin_igv': 38385.18, 'gastos_operativos': 4681.12, 'mantenimiento_con_igv': 46811.2, 'mantenimiento_con_go': 51492.32, 'igv': 8426.02}, '12': {'mantenimiento_sin_igv': 1140.58, 'gastos_operativos': 139.09, 'mantenimiento_con_igv': 1390.95, 'mantenimiento_con_go': 1530.05, 'igv': 250.37}, '4': {'mantenimiento_sin_igv': 1824.92, 'gas

In [135]:
cronograma_desembolsos_current= cronograma_desembolsos.get(str(mes_recursos_solicitados), {})
if not cronograma_desembolsos_current:
    print(f"No se encontraron datos de costos para el mes {mes_recursos_solicitados}.")
print(f"Costo de mantenimiento para el mes {mes_recursos_solicitados}:", cronograma_desembolsos_current)

Costo de mantenimiento para el mes 5: {'mantenimiento_sin_igv': 40127.65, 'gastos_operativos': 4893.62, 'mantenimiento_con_igv': 48936.16, 'mantenimiento_con_go': 53829.78, 'igv': 8808.51}


In [136]:
costo_mantenimiento_rutinario=cronograma_desembolsos_current['mantenimiento_con_igv']
costo_gastos_operativos=cronograma_desembolsos_current['gastos_operativos']
costo_total_mantenimiento=cronograma_desembolsos_current['mantenimiento_con_go']
print(f"Costo de mantenimiento rutinario: {costo_mantenimiento_rutinario}")
print(f"Costo de gastos operativos: {costo_gastos_operativos}")
print(f"Costo total de mantenimiento: {costo_total_mantenimiento}")

Costo de mantenimiento rutinario: 48936.16
Costo de gastos operativos: 4893.62
Costo total de mantenimiento: 53829.78


In [137]:
proyecto_snapshot=(
    db
    .collection(colec_raiz)
    .document(doc_proyecto)
).get()

if not proyecto_snapshot.exists:
    print(f"El documento de datos generales no existe.")
proyecto_from_firestore = proyecto_snapshot.to_dict()
print(proyecto_from_firestore)

{'otros': {'tipo_superficie': 'Asfaltado', 'estado_conservacion_via': 'regular'}, 'expediente': {'codigo_ruta': 'AR-683', 'monto_total': 206413.0, 'coordenadas': {'fin': {'hemisferio': 'S', 'y': 8270075.15, 'progresiva': 32252.0, 'zona_letra': None, 'x': 220383.97, 'datum': 'WGS84', 'altitud': 3629, 'zona': None}, 'inicio': {'hemisferio': 'S', 'y': 8281146.79, 'progresiva': 0, 'zona_letra': None, 'x': 226316.6, 'datum': 'WGS84', 'altitud': 3820, 'zona': None}}, 'nombre': 'MANTENIMIENTO VIAL RUTINARIO CAMINO VECINAL EMP. AR-111 (NUEVO SIBAYO) - TUTI - EMP.AR-681 (DV. CHIVAY), DISTRITOS DE CHIVAY, TUTI Y SIBAYO, PROVINCIA DE CAYLLOMA, DEPARTAMENTO DE AREQUIPA', 'tarifa_km_mes': 800, 'longitud': 32252.0}, 'other': {'tipo_superficie': 'Asfaltado', 'estado_conservacion_via': 'regular'}, 'datos_generales': {'distritos': ['Chivay', 'Tuti', 'Sibayo'], 'region': 'Arequipa', 'provincia': 'Caylloma'}, 'contrato': {'jefe_mantenimiento': {'titulo': 'Ingeniero', 'nombre': 'Genaro', 'dni': 0, 'apelli

In [138]:
datos_generales = proyecto_from_firestore.get("datos_generales", {})
if not datos_generales:
    print("No se encontraron datos generales en el documento.")
print("Datos generales:", datos_generales)

Datos generales: {'distritos': ['Chivay', 'Tuti', 'Sibayo'], 'region': 'Arequipa', 'provincia': 'Caylloma'}


In [139]:
distritos, region, provincia = '-'.join(datos_generales['distritos']), datos_generales['region'], datos_generales['provincia']
print(f"Distritos: {distritos}, Región: {region}, Provincia: {provincia}")

Distritos: Chivay-Tuti-Sibayo, Región: Arequipa, Provincia: Caylloma


In [140]:
expediente= proyecto_from_firestore.get("expediente", {})
if not expediente:
    print("No se encontraron datos del expediente en el documento.")

print("Datos del expediente:", expediente)

Datos del expediente: {'codigo_ruta': 'AR-683', 'monto_total': 206413.0, 'coordenadas': {'fin': {'hemisferio': 'S', 'y': 8270075.15, 'progresiva': 32252.0, 'zona_letra': None, 'x': 220383.97, 'datum': 'WGS84', 'altitud': 3629, 'zona': None}, 'inicio': {'hemisferio': 'S', 'y': 8281146.79, 'progresiva': 0, 'zona_letra': None, 'x': 226316.6, 'datum': 'WGS84', 'altitud': 3820, 'zona': None}}, 'nombre': 'MANTENIMIENTO VIAL RUTINARIO CAMINO VECINAL EMP. AR-111 (NUEVO SIBAYO) - TUTI - EMP.AR-681 (DV. CHIVAY), DISTRITOS DE CHIVAY, TUTI Y SIBAYO, PROVINCIA DE CAYLLOMA, DEPARTAMENTO DE AREQUIPA', 'tarifa_km_mes': 800, 'longitud': 32252.0}


In [141]:
codigo_ruta= expediente.get("codigo_ruta", "")
if not codigo_ruta:
    print("No se encontró el código de ruta en los datos del expediente.")
print(f"Código de ruta: {codigo_ruta}")

Código de ruta: AR-683


In [142]:
longitud = expediente.get("longitud", 0)
if longitud is None:
    print("No se encontró la longitud en los datos del expediente.")
print(f"Longitud: {longitud} km")

Longitud: 32252.0 km


In [143]:
contrato= proyecto_from_firestore.get("contrato", {})
if not contrato:
    print("No se encontraron datos del contrato en el documento.")
print("Datos del contrato:", contrato)

Datos del contrato: {'jefe_mantenimiento': {'titulo': 'Ingeniero', 'nombre': 'Genaro', 'dni': 0, 'apellido': 'Tinta Cáceres'}, 'numero_trabajadores': 3, 'numero_cuadrillas': 1, 'tipo_servicio': 'mantenimiento rutinario', 'denominacion_tramo_convenio': 'EMP. AR-111 (NUEVO SIBAYO) - TUTI EMP. AR-681 (DV. CHIVAY) (KM 32+252)', 'fecha_inicio': {'anio': 2025, 'mes': 4, 'dia': 15}, 'id_contrato': '004-2025', 'monto_contrato': 165130.4, 'contratista': {'razon_social': 'PLATERS MANAGEMENT S.A.C.', 'ruc': ''}, 'tiempo_ejecucion_dias': 240}


In [144]:
denominacion_tramo_convenio= contrato.get("denominacion_tramo_convenio", "")
if not denominacion_tramo_convenio:
    print("No se encontró la denominación del tramo en el convenio.")
print(f"Denominación del tramo en el convenio: {denominacion_tramo_convenio}")

Denominación del tramo en el convenio: EMP. AR-111 (NUEVO SIBAYO) - TUTI EMP. AR-681 (DV. CHIVAY) (KM 32+252)


In [145]:
data_for_document={
  "departamento": region,
  "provincia": provincia,
  "distrito": distritos,
  "tramo": denominacion_tramo_convenio,
  "codigo_ruta": codigo_ruta,
  "categoria": "Camino Vecinal",
  "longitud_km": str(round(longitud/1000, 2)),
  "mes": calendar.month_name[mes_recursos_solicitados],
  "mantenimiento_rutinario": costo_mantenimiento_rutinario,
  "gastos_operativos": costo_gastos_operativos,
  "costo_total_mantenimiento": costo_total_mantenimiento,
}

In [146]:
print(data_for_document)

{'departamento': 'Arequipa', 'provincia': 'Caylloma', 'distrito': 'Chivay-Tuti-Sibayo', 'tramo': 'EMP. AR-111 (NUEVO SIBAYO) - TUTI EMP. AR-681 (DV. CHIVAY) (KM 32+252)', 'codigo_ruta': 'AR-683', 'categoria': 'Camino Vecinal', 'longitud_km': '32.25', 'mes': 'Mayo', 'mantenimiento_rutinario': 48936.16, 'gastos_operativos': 4893.62, 'costo_total_mantenimiento': 53829.78}


In [147]:
# —————————————————————————————————————————————
# Filtro para formato de moneda
# —————————————————————————————————————————————
def format_currency(value):
    return f"{value:,.2f}"

# —————————————————————————————————————————————
# Plantilla HTML + CSS (A4 landscape) con tu estructura de tabla
# —————————————————————————————————————————————
html_template = """
<!DOCTYPE html>
<html lang="es">
<head>
  <meta charset="UTF-8"/>
  <style>
    @page { size: A4 landscape; margin: 1cm; }
    body { font-family: sans-serif; font-size: 11pt; }
    .title { text-align: center; font-size: 16pt; font-weight: bold; margin-bottom: 8px; }
    table { width: 100%; border-collapse: collapse; }
    .header-table td { padding: 4px 6px; vertical-align: top; }
    .content-table th {
      background-color: #2C3E50; color: white; padding: 6px; text-align: left; font-size: 10pt;
    }
    .content-table td {
      border: 1px solid #bbb; padding: 6px; vertical-align: top;
      word-break: break-word;
    }
    .content-table tr:nth-child(even) { background-color: #f2f2f2; }
    .footer-row td {
      font-weight: bold; background-color: #dde4ec;
    }
  </style>
</head>
<body>

  <div class="title">Requerimiento de recursos</div>

  <!-- Datos superiores, sin bordes -->
  <table class="header-table">
    <tr>
      <td><strong>CAMINO:</strong> {{ tramo }}</td>
      <td><strong>CATEGORÍA:</strong> {{ categoria }}</td>
      <td><strong>CÓDIGO DE RUTA:</strong> {{ codigo_ruta }}</td>
      <td><strong>LONGITUD TOTAL:</strong> {{ longitud_km }} km</td>
    </tr>
  </table>

  <!-- Tabla principal con rowspan/colspan -->
  <table class="content-table">
    <thead>
      <tr>
        <th rowspan="2">DEPARTAMENTO</th>
        <th rowspan="2">PROVINCIA</th>
        <th rowspan="2">DISTRITO</th>
        <th rowspan="2">TRAMO</th>
        <th colspan="3">{{ mes.upper() }}</th>
      </tr>
      <tr>
        <th>MANTENIMIENTO RUTINARIO</th>
        <th>GASTOS OPERATIVOS</th>
        <th>TOTAL</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>{{ departamento }}</td>
        <td>{{ provincia }}</td>
        <td>{{ distrito }}</td>
        <td>{{ tramo }}</td>
        <td>{{ mantenimiento_rutinario | format_currency }}</td>
        <td>{{ gastos_operativos | format_currency }}</td>
        <td>{{ costo_total_mantenimiento | format_currency }}</td>
      </tr>
      <tr class="footer-row">
        <td colspan="4"><strong>TOTAL</strong></td>
        <td><strong>{{ mantenimiento_rutinario | format_currency }}</strong></td>
        <td><strong>{{ gastos_operativos | format_currency }}</strong></td>
        <td><strong>{{ costo_total_mantenimiento | format_currency }}</strong></td>
      </tr>
    </tbody>
  </table>

</body>
</html>
"""

# —————————————————————————————————————————————
# Crear Environment y registrar filtro
# —————————————————————————————————————————————
env = Environment()
env.filters['format_currency'] = format_currency
template = env.from_string(html_template)

# —————————————————————————————————————————————
# Renderizar HTML y generar PDF con xhtml2pdf
# —————————————————————————————————————————————
html_out = template.render(**data_for_document)

ruta_directorio = os.path.join("output", proyecto, str(mes))
ruta_archivo_requerimiento_recursos = os.path.join(ruta_directorio, "requerimiento_recursos.pdf")

# Crear el directorio si no existe
os.makedirs(ruta_directorio, exist_ok=True)
# Guardar el diccionario en un archivo JSON

def html_to_pdf(source_html: str, output_filename: str):
    with open(output_filename, "wb") as output_file:
        pisa_status = pisa.CreatePDF(source_html, dest=output_file)
    return pisa_status.err

err = html_to_pdf(html_out, ruta_archivo_requerimiento_recursos)
if err:
    print("⚠️ Ocurrió un error al generar el PDF.")
else:
    print("✔️ PDF generado: requerimiento_de_recursos.pdf")

✔️ PDF generado: requerimiento_de_recursos.pdf
